 # 🚀 Final Training: ResNet50 + Augmented Data
 זהו הסקריפט הסופי לאימון המודל.
 הוא כולל:
 1. מודל מנצח: ResNet50 Dual-Head.
 2. פונקציות הפסד: BCE (למפה) + MSE (לווקטורים).
 3. דאטה מלא: כל תמונות האימון.
 4. אוגמנטציות חכמות: היפוך (כולל תיקון וקטורים), צבע, טשטוש וסיבוב.

In [2]:
import os
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
import time
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from torchvision.transforms import functional as TF
from pathlib import Path
from tqdm import tqdm

# --- הגדרות סופיות ---
BATCH_SIZE = 4          # ResNet50 הוא כבד, נוריד ל-4 כדי לא לפוצץ זיכרון
LEARNING_RATE = 0.0001
NUM_EPOCHS = 20         # אימון ארוך ומלא
DATA_DIR = Path("DATA")
SAVE_DIR = Path("final_models")
SAVE_DIR.mkdir(exist_ok=True)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"--- Final Training Setup ---")
print(f"Device: {DEVICE}")
print(f"Backbone: ResNet50")
print(f"Augmentations: ON (Flip, Color, Rotate +/-5, Blur)")

--- Final Training Setup ---
Device: cuda
Backbone: ResNet50
Augmentations: ON (Flip, Color, Rotate +/-5, Blur)


# 1. Smart Augmented Dataset
 מחלקה שטוענת את הדאטה ומפעילה עליו שינויים רנדומליים בזמן אמת.

In [3]:
class AugmentedRoadDataset(Dataset):
    def __init__(self, root_dir, split='train', target_size=(512, 256)):
        self.root_dir = Path(root_dir)
        self.split = split
        self.target_size = target_size
        
        self.img_dir = self.root_dir / "images" / split
        self.mask_dir = self.root_dir / "masks" / split
        self.flow_dir = self.root_dir / "flow" / split
        
        self.images = sorted(list(self.img_dir.glob("*.jpg")))
        
        # אוגמנטציות לצבע בלבד (לא משפיע על גיאומטריה)
        self.color_jit = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
        self.blur = transforms.GaussianBlur(kernel_size=3)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # 1. טעינה
        stem = self.images[idx].stem
        img = cv2.imread(str(self.images[idx]))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(str(self.mask_dir / f"{stem}.png"), cv2.IMREAD_GRAYSCALE)
        flow = np.load(str(self.flow_dir / f"{stem}.npy")) # (H, W, 2)

        # 2. Resize אחיד לכולם
        img = cv2.resize(img, self.target_size)
        mask = cv2.resize(mask, self.target_size, interpolation=cv2.INTER_NEAREST)
        flow = cv2.resize(flow, self.target_size)

        # המרה ל-PIL/Tensor לצורך אוגמנטציות של Torchvision
        img_pil = TF.to_pil_image(img)
        mask_pil = TF.to_pil_image(mask)
        flow_tensor = torch.from_numpy(flow).permute(2, 0, 1) # (2, H, W)

        # --- 3. אוגמנטציות (רק ל-TRAIN) ---
        if self.split == 'train':
            # A. שינויי צבע וטשטוש (רק על התמונה)
            if random.random() > 0.5:
                img_pil = self.color_jit(img_pil)
            if random.random() > 0.8: # לעיתים רחוקות יותר
                img_pil = self.blur(img_pil)

            # B. היפוך אופקי (Horizontal Flip) - דורש טיפול מיוחד
            if random.random() > 0.5:
                img_pil = TF.hflip(img_pil)
                mask_pil = TF.hflip(mask_pil)
                flow_tensor = TF.hflip(flow_tensor)
                # הקסם: אם הופכים תמונה, וקטור ימינה הופך לוקטור שמאלה
                # לכן חייבים להפוך את הסימן של רכיב ה-X (ערוץ 0)
                flow_tensor[0, :, :] = -flow_tensor[0, :, :]

            # C. סיבוב קל (Rotation +/- 5 degrees)
            if random.random() > 0.5:
                angle = random.uniform(-5, 5)
                # מסובבים את כולם באותה זווית בדיוק
                img_pil = TF.rotate(img_pil, angle)
                mask_pil = TF.rotate(mask_pil, angle)
                flow_tensor = TF.rotate(flow_tensor, angle)
                # הערה: בסיבוב קטן אנחנו מזניחים את סיבוב ערכי הוקטור עצמם

        # 4. המרה סופית ל-Tensor ונרמול
        img_tensor = TF.to_tensor(img_pil) # מנרמל אוטומטית ל-0-1
        mask_tensor = TF.to_tensor(mask_pil)
        
        # ה-Flow כבר בטנזור, רק לוודא שהוא Float
        flow_tensor = flow_tensor.float()

        return img_tensor, mask_tensor, flow_tensor


# 2. ResNet50 Dual-Head Model
 המודל הגדול והחזק.

In [4]:
class DualHeadResNet50(nn.Module):
    def __init__(self):
        super().__init__()
        # טוענים ResNet50 מאומן מראש
        base = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        layers = list(base.children())
        
        # Encoder
        self.layer0 = nn.Sequential(*layers[:3])
        self.layer1 = nn.Sequential(*layers[3:5])
        self.layer2 = layers[5]
        self.layer3 = layers[6]
        self.layer4 = layers[7]
        
        # Decoder (מותאם לערוצים של ResNet50: 2048, 1024, 512, 256)
        self.up4 = self._up_block(2048, 1024)
        self.up3 = self._up_block(1024 + 1024, 512)
        self.up2 = self._up_block(512 + 512, 256)
        self.up1 = self._up_block(256 + 256, 64)
        self.up0 = self._up_block(64 + 64, 32)
        
        # Heads
        self.potential_head = nn.Sequential(nn.Conv2d(32, 1, 1), nn.Sigmoid())
        self.flow_head = nn.Sequential(nn.Conv2d(32, 2, 1), nn.Tanh())

    def _up_block(self, in_c, out_c):
        return nn.Sequential(
            nn.ConvTranspose2d(in_c, out_c, 2, 2),
            nn.BatchNorm2d(out_c), nn.ReLU(inplace=True),
            nn.Conv2d(out_c, out_c, 3, padding=1),
            nn.BatchNorm2d(out_c), nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x0 = self.layer0(x)
        x1 = self.layer1(x0)
        x2 = self.layer2(x1)
        x3 = self.layer3(x2)
        x4 = self.layer4(x3)
        
        up4 = self.up4(x4)
        up3 = self.up3(torch.cat([up4, x3], 1))
        up2 = self.up2(torch.cat([up3, x2], 1))
        up1 = self.up1(torch.cat([up2, x1], 1))
        feat = self.up0(torch.cat([up1, x0], 1))
        
        return self.potential_head(feat), self.flow_head(feat)


#  3. Training Engine
 לולאת האימון המלאה עם שמירת מדדים.

In [5]:
def calculate_metrics(pred_pot, target_pot, pred_flow, target_flow):
    # Segmentation Metrics (IoU, Precision, Recall)
    pred_bin = (pred_pot > 0.5).float()
    target_bin = (target_pot > 0.5).float()
    
    tp = (pred_bin * target_bin).sum().item()
    fp = (pred_bin * (1-target_bin)).sum().item()
    fn = ((1-pred_bin) * target_bin).sum().item()
    
    epsilon = 1e-6
    iou = tp / (tp + fp + fn + epsilon)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    
    # Flow Metrics (EPE)
    diff = pred_flow - target_flow
    epe = torch.norm(diff, dim=1).mean().item()
    
    return iou, precision, recall, epe

def train_model():
    # 1. Data Loaders (Full Dataset)
    train_ds = AugmentedRoadDataset(DATA_DIR, 'train')
    val_ds = AugmentedRoadDataset(DATA_DIR, 'val')
    
    # num_workers=2 מאיץ את הטעינה (בזהירות עם ווינדוס)
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    print(f"Train Images: {len(train_ds)} | Val Images: {len(val_ds)}")
    
    # 2. Model Setup
    model = DualHeadResNet50().to(DEVICE)
    optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    # 3. Losses (The Winners)
    criterion_pot = nn.BCELoss() # Best for Map
    criterion_flow = nn.MSELoss() # Best for Vector Length & Direction combined
    
    best_val_loss = float('inf')
    history = []
    
    # 4. Main Loop
    for epoch in range(NUM_EPOCHS):
        print(f"\n--- Epoch {epoch+1}/{NUM_EPOCHS} ---")
        current_lr = optimizer.param_groups[0]['lr']
        print(f"LR: {current_lr}")
        
        # Train Step
        model.train()
        train_loss = 0
        loop = tqdm(train_loader, desc="Training", leave=False)
        
        for imgs, masks, flows in loop:
            imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
            
            # Forward
            pred_pot, pred_flow = model(imgs)
            
            # Loss
            loss_pot = criterion_pot(pred_pot, masks)
            loss_flow = criterion_flow(pred_flow, flows)
            loss = loss_pot + loss_flow
            
            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation Step
        model.eval()
        val_loss = 0
        total_iou = 0
        total_prec = 0
        total_rec = 0
        total_epe = 0
        
        with torch.no_grad():
            for imgs, masks, flows in val_loader:
                imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
                pred_pot, pred_flow = model(imgs)
                
                # Loss
                loss_pot = criterion_pot(pred_pot, masks)
                loss_flow = criterion_flow(pred_flow, flows)
                val_loss += (loss_pot + loss_flow).item()
                
                # Metrics
                iou, prec, rec, epe = calculate_metrics(pred_pot, masks, pred_flow, flows)
                total_iou += iou
                total_prec += prec
                total_rec += rec
                total_epe += epe
        
        avg_val_loss = val_loss / len(val_loader)
        avg_iou = total_iou / len(val_loader)
        avg_prec = total_prec / len(val_loader)
        avg_rec = total_rec / len(val_loader)
        avg_epe = total_epe / len(val_loader)
        
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val Loss:   {avg_val_loss:.4f} | IoU: {avg_iou:.3f} | Prec: {avg_prec:.3f} | Rec: {avg_rec:.3f} | EPE: {avg_epe:.3f}")
        
        # Scheduler Step
        scheduler.step(avg_val_loss)
        
        # Save Best
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), SAVE_DIR / "final_model_r50_best.pth")
            print(">>> 💾 New Best Model Saved!")
            
        # Log History
        history.append({
            "Epoch": epoch+1,
            "Train Loss": avg_train_loss,
            "Val Loss": avg_val_loss,
            "IoU": avg_iou,
            "Precision": avg_prec,
            "Recall": avg_rec,
            "EPE": avg_epe,
            "LR": current_lr
        })
        
    # Save Final Report
    pd.DataFrame(history).to_csv(SAVE_DIR / "training_history.csv", index=False)
    print("\n✅ Final Training Complete. Model saved in 'final_models/'.")

if __name__ == "__main__":
    train_model()

Train Images: 6906 | Val Images: 979

--- Epoch 1/20 ---
LR: 0.0001


Train Loss: 0.3814
Val Loss:   0.2434 | IoU: 0.776 | Prec: 0.812 | Rec: 0.944 | EPE: 0.131
>>> 💾 New Best Model Saved!

--- Epoch 2/20 ---
LR: 0.0001


Train Loss: 0.2127
Val Loss:   0.2096 | IoU: 0.775 | Prec: 0.932 | Rec: 0.817 | EPE: 0.111
>>> 💾 New Best Model Saved!

--- Epoch 3/20 ---
LR: 0.0001


Train Loss: 0.1951
Val Loss:   0.2041 | IoU: 0.792 | Prec: 0.923 | Rec: 0.845 | EPE: 0.104
>>> 💾 New Best Model Saved!

--- Epoch 4/20 ---
LR: 0.0001


Train Loss: 0.1908
Val Loss:   0.2004 | IoU: 0.823 | Prec: 0.909 | Rec: 0.894 | EPE: 0.102
>>> 💾 New Best Model Saved!

--- Epoch 5/20 ---
LR: 0.0001


...

Train Loss: 0.1816
Val Loss:   0.1987 | IoU: 0.827 | Prec: 0.888 | Rec: 0.921 | EPE: 0.089

--- Epoch 17/20 ---
LR: 0.0001


Train Loss: 0.1811
Val Loss:   0.1981 | IoU: 0.832 | Prec: 0.894 | Rec: 0.920 | EPE: 0.090

--- Epoch 18/20 ---
LR: 5e-05


Train Loss: 0.1794
Val Loss:   0.1975 | IoU: 0.834 | Prec: 0.880 | Rec: 0.938 | EPE: 0.084
>>> 💾 New Best Model Saved!

--- Epoch 19/20 ---
LR: 5e-05


Train Loss: 0.1783
Val Loss:   0.1966 | IoU: 0.841 | Prec: 0.910 | Rec: 0.914 | EPE: 0.082
>>> 💾 New Best Model Saved!

--- Epoch 20/20 ---
LR: 5e-05


Train Loss: 0.1776
Val Loss:   0.1972 | IoU: 0.829 | Prec: 0.913 | Rec: 0.897 | EPE: 0.087

✅ Final Training Complete. Model saved in 'final_models/'.


In [ ]:
# --- הגדרות להמשך אימון (Fine-Tuning) ---
EXTENDED_EPOCHS = 20
FINE_TUNE_LR = 1e-5  # קצב למידה עדין מאוד (פי 10 פחות מהמקורי)

def train_extended_session():
    print(f"\n🚀 Starting Extended Training (Fine-Tuning) on {DEVICE}")
    print(f"   Target LR: {FINE_TUNE_LR} | Epochs: {EXTENDED_EPOCHS}")

    # 1. טעינת הדאטה (שימוש במחלקות הקיימות)
    train_ds = AugmentedRoadDataset(DATA_DIR, 'train')
    val_ds = AugmentedRoadDataset(DATA_DIR, 'val')
    
    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
    
    # 2. יצירת המודל וטעינת המשקולות הכי טובות מהריצה הקודמת
    model = DualHeadResNet50().to(DEVICE)
    best_model_path = SAVE_DIR / "final_model_r50_best.pth"
    
    if best_model_path.exists():
        print(f"   Loading weights from: {best_model_path}")
        model.load_state_dict(torch.load(best_model_path, map_location=DEVICE))
    else:
        print(f"❌ Error: {best_model_path} not found! Run the main training first.")
        return

    # 3. הגדרת אופטימייזר עדין
    optimizer = optim.AdamW(model.parameters(), lr=FINE_TUNE_LR, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)
    
    criterion_pot = nn.BCELoss()
    criterion_flow = nn.MSELoss()
    
    # מאפסים את ה-Best Loss כדי לשמור שיפורים מקומיים של הסשן הזה
    best_val_loss = float('inf')
    history = []

    # 4. לולאת האימון הנוספת
    for epoch in range(EXTENDED_EPOCHS):
        print(f"\n--- Extended Epoch {epoch+1}/{EXTENDED_EPOCHS} ---")
        print(f"LR: {optimizer.param_groups[0]['lr']}")
        
        # Train
        model.train()
        train_loss = 0
        loop = tqdm(train_loader, desc="Fine-Tuning", leave=False)
        
        for imgs, masks, flows in loop:
            imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
            
            pot_out, flow_out = model(imgs)
            loss = criterion_pot(pot_out, masks) + criterion_flow(flow_out, flows)
            
            optimizer.zero_grad(); loss.backward(); optimizer.step()
            train_loss += loss.item()
            loop.set_postfix(loss=loss.item())
            
        avg_train_loss = train_loss / len(train_loader)
        
        # Validation
        model.eval()
        val_loss = 0
        total_iou = 0
        total_prec = 0
        total_rec = 0
        total_epe = 0
        
        with torch.no_grad():
            for imgs, masks, flows in val_loader:
                imgs, masks, flows = imgs.to(DEVICE), masks.to(DEVICE), flows.to(DEVICE)
                pot_out, flow_out = model(imgs)
                
                val_loss += (criterion_pot(pot_out, masks) + criterion_flow(flow_out, flows)).item()
                iou, prec, rec, epe = calculate_metrics(pot_out, masks, flow_out, flows)
                
                total_iou += iou
                total_prec += prec
                total_rec += rec
                total_epe += epe
        
        avg_val_loss = val_loss / len(val_loader)
        metrics = [
            total_iou / len(val_loader),
            total_prec / len(val_loader),
            total_rec / len(val_loader),
            total_epe / len(val_loader)
        ]
        
        print(f"Train Loss: {avg_train_loss:.4f}")
        print(f"Val Loss:   {avg_val_loss:.4f} | IoU: {metrics[0]:.3f} | Prec: {metrics[1]:.3f} | Rec: {metrics[2]:.3f} | EPE: {metrics[3]:.3f}")
        
        scheduler.step(avg_val_loss)
        
        # שומרים בשם חדש (_extended)
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), SAVE_DIR / "final_model_r50_extended.pth")
            print(">>> 💾 Extended Best Model Saved!")
            
        history.append({
            "Epoch": f"Ext-{epoch+1}",
            "Train Loss": avg_train_loss,
            "Val Loss": avg_val_loss,
            "IoU": metrics[0],
            "EPE": metrics[3]
        })

    # שומרים את ההיסטוריה לקובץ נפרד
    pd.DataFrame(history).to_csv(SAVE_DIR / "training_history_extended.csv", index=False)
    print("\n✅ Fine-Tuning Complete. Model saved as 'final_model_r50_extended.pth'.")

if __name__ == "__main__":
    train_extended_session()


🚀 Starting Extended Training (Fine-Tuning) on cuda
   Target LR: 1e-05 | Epochs: 20
   Loading weights from: final_models\final_model_r50_best.pth

--- Extended Epoch 1/20 ---
LR: 1e-05


Train Loss: 0.1766
Val Loss:   0.1974 | IoU: 0.836 | Prec: 0.905 | Rec: 0.912 | EPE: 0.086
>>> 💾 Extended Best Model Saved!

--- Extended Epoch 2/20 ---
LR: 1e-05


Fine-Tuning:  73%|███████▎  | 1257/1727 [07:15<02:52,  2.72it/s, loss=0.12] 